In [3]:
# import random

# import youtokentome as yttm

# train_data_path = "train_data.txt"
# model_path = "example.model"

# # Generating random file with training data
# # 10000 lines with 100 characters in each line
# n_lines = 10000
# n_characters = 100
# with open(train_data_path, "w") as fout:
#     for _ in range(n_lines):
#         print("".join([random.choice("abcd ") for _ in range(n_characters)]), file=fout)

# # Generating random text
# test_text = "".join([random.choice("abcde ") for _ in range(100)])

# # Training model
# yttm.BPE.train(data=train_data_path, vocab_size=5000, model=model_path)

# # Loading model
# bpe = yttm.BPE(model=model_path)

# # Two types of tokenization
# print(bpe.encode([test_text], output_type=yttm.OutputType.ID, dropout_prob = 0.7))
# print(bpe.encode([test_text], output_type=yttm.OutputType.SUBWORD))

In [2]:
from transformers import RobertaTokenizer
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4

import json 

import spacy

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [5]:
# from pathlib import Path

# from tokenizers import ByteLevelBPETokenizer

# paths = [str(x) for x in Path("./").glob("**/*.txt")]

# # Initialize a tokenizer
# tokenizer = ByteLevelBPETokenizer()

# # Customize training
# tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
#     "[s1]",
#     "[s2]",
#     "[e1]",
#     "[e2]"
# ])

# # Save files to disk
# tokenizer.save_model(".", "pah")
# tokenizer.save("pah/tokenizer/tokenizer_file")

In [6]:
# from tokenizers.implementations import ByteLevelBPETokenizer
# from tokenizers.processors import BertProcessing


# tokenizer = ByteLevelBPETokenizer(
#     "pah/tokenizer/vocab.json",
#     "pah/tokenizer/merges.txt",
# )
# tokenizer._tokenizer.post_processor = BertProcessing(
#     ("</s>", tokenizer.token_to_id("</s>")),
#     ("<s>", tokenizer.token_to_id("<s>")),
# )
# tokenizer.enable_truncation(max_length=512)

# print(
#     tokenizer.encode("6-minutes walk distance")
# )

# tokenizer.save("pah/tokenizer/tokenizer_file")

In [7]:
# tokenizer.tokenize()

In [3]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52000,
    max_position_embeddings=600,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [4]:
from transformers import RobertaTokenizerFast,PreTrainedTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("pah/tokenizer", max_len=768)
# tokenizer = PreTrainedTokenizerFast(tokenizer_file="pah/tokenizer/tokenizer_file")


In [5]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification(config=config)

In [11]:
model.num_parameters()


83518466

In [12]:
import torch
torch.cuda.is_available()

True

In [13]:
import datasets

In [7]:
train_data = pd.read_csv('pah/data/train_fs.tsv', delimiter = '\t', header = None)
test_data = pd.read_csv('pah/data/dev_fs.tsv', delimiter = '\t', header = None)

In [8]:
train_data.head()

,0,1,2,3,4,5,6
0,Rel,"Methods In this double-blind , placebo-control...",dosage,drug,T_133115,T_13350,F_133
1,NotRel,"In a preliminary study , the orally administer...",drug,reason,T_1331,T_133113,F_133
2,Rel,"Methods In this double-blind , placebo-control...",drug,duration,T_1335,T_1331161,F_133
3,Rel,"Methods In this double-blind , placebo-control...",drug,dosage,T_1336,T_1331172,F_133
4,Rel,"Methods In this double-blind , placebo-control...",drug,duration,T_1336,T_1331183,F_133


In [9]:
train_data = train_data[[0,1]]
train_data.columns = ['label','utterance']

test_data = test_data[[0,1]]
test_data.columns = ['label','utterance']

In [10]:
def prepare_features(seq_1, max_seq_length = 300, 
             zero_pad = True, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    # input_ids = input_ids.squeeze(0)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [11]:
class Relations(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        utterance = self.data.utterance[index]
        label = self.data.label[index]
        X, _  = prepare_features(utterance)
        y = label_to_ix[self.data.label[index]]
        return X,y #{k: v[index] for k, v in zip(X,y)}#
    
    def __len__(self):
        return self.len

In [12]:
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification(config)

In [59]:
training_set = Relations(train_data)
testing_set = Relations(test_data)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.cuda()

In [53]:
params = {'batch_size': 1,
          'shuffle': True,
          'drop_last': False,
          'num_workers': 1}

In [60]:
training_loader = DataLoader(training_set, **params)
testing_loader = DataLoader(testing_set, **params)

In [17]:
loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05
optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)

In [18]:
label_to_ix = {}
for label in train_data.label:
    for word in label.split():
        if word not in label_to_ix:
            label_to_ix[word]=len(label_to_ix)
label_to_ix

{'Rel': 0, 'NotRel': 1}

In [27]:
config = RobertaConfig.from_pretrained('roberta-base')
config.num_labels = len(list(label_to_ix.values()))
config

RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [19]:
for param in list(model.parameters())[-90:]: # total  trainable 199 Params: 79 is 40%
    param.requires_grad = False

In [20]:
len(list(model.parameters()))

105

In [21]:
## Test Forward Pass
inp = training_set.__getitem__(0)[0].cuda()
output = model(inp)[0]
print(inp.shape, output.shape)

torch.Size([1, 300]) torch.Size([1, 2])


In [22]:
inp.size()

torch.Size([1, 300])

In [23]:
training_set.__getitem__(28)[0].shape

torch.Size([1, 300])

In [24]:
model(training_set.__getitem__(29)[0].cuda())

SequenceClassifierOutput(loss=None, logits=tensor([[0.1558, 0.5354]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
len(train_data.loc[2,'utterance'])

391

In [63]:
max_epochs = 10
model = model.train()
latest_best_score = 0
for epoch in tqdm_notebook(range(max_epochs)):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader):
        optimizer.zero_grad()
        sent = sent.squeeze(1)
        if torch.cuda.is_available():
          sent = sent.cuda()
          label = label.cuda()
        output = model.forward(sent)[0]
        _, predicted = torch.max(output, 1)
        
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        if i%1000 == 0:
            correct = 0
            total = 0
            for sent, label in testing_loader:
                sent = sent.squeeze(0)
                if torch.cuda.is_available():
                  sent = sent.cuda()
                  label = label.cuda()
                output = model.forward(sent)[0]
                _, predicted = torch.max(output.data, 1)
                total += label.size(0)
                correct += (predicted.cpu() == label.cpu()).sum()
            accuracy = 100.00 * correct.numpy() / total
            if accuracy>latest_best_score and accuracy>0.7:
                latest_best_score = accuracy
                torch.save(model.state_dict(), 'model_roberta_vars_'+str(epoch)+'_'+ str(uuid4())+'.pth')

            print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/10 [00:00<?, ?it/s]

EPOCH -- 0
Iteration: 0. Loss: 0.9723549485206604. Accuracy: 65.64327485380117%
Iteration: 1000. Loss: 0.30841875076293945. Accuracy: 65.64327485380117%
Iteration: 2000. Loss: 0.442291259765625. Accuracy: 65.64327485380117%
Iteration: 3000. Loss: 0.9026875495910645. Accuracy: 65.64327485380117%
Iteration: 4000. Loss: 0.3848063349723816. Accuracy: 65.64327485380117%
Iteration: 5000. Loss: 1.2698019742965698. Accuracy: 65.64327485380117%
Iteration: 6000. Loss: 1.0124282836914062. Accuracy: 65.64327485380117%
EPOCH -- 1
Iteration: 0. Loss: 0.34892404079437256. Accuracy: 65.64327485380117%
Iteration: 1000. Loss: 0.3699212670326233. Accuracy: 65.64327485380117%
Iteration: 2000. Loss: 0.3944622576236725. Accuracy: 65.64327485380117%
Iteration: 3000. Loss: 0.33682170510292053. Accuracy: 65.64327485380117%
Iteration: 4000. Loss: 0.25837451219558716. Accuracy: 65.64327485380117%
Iteration: 5000. Loss: 0.3808467984199524. Accuracy: 65.64327485380117%
Iteration: 6000. Loss: 1.3068909645080566. Ac

In [27]:
train_jsons_aug = json.load(open('pah/data/relations_marked_train_aug.json'))
len(train_jsons_aug)

983

In [28]:
test_jsons_aug = json.load(open('pah/data/relations_marked_test_aug.json'))
len(test_jsons_aug)

226

In [29]:
inf_jsons_aug = json.load(open('pah/data/articles_inf_aa_aug.json'))


In [30]:
ent_dict = {}
for i in range(len(train_jsons_aug)):
    for ent in  train_jsons_aug[i]['entities']:
        if ent['type'] not in ent_dict:
            ent_dict[ent['type']] = len(ent_dict)
            
for i in range(len(test_jsons_aug)):
    for ent in  test_jsons_aug[i]['entities']:
        if ent['type'] not in ent_dict:
            ent_dict[ent['type']] = len(ent_dict)
            
for i in range(len(inf_jsons_aug)):
    for ent in  inf_jsons_aug[i]['entities']:
        if ent['type'] not in ent_dict:
            ent_dict[ent['type']] = len(ent_dict)
ent_dict['O'] = len(ent_dict)

In [31]:
TAG_LIST = [".",",",":", "-LRB-","-RRB-","``","\"\"","''",",","$","#","AFX","CC","CD","DT","EX","FW","HYPH","IN","JJ","JJR","JJS","LS","MD","NIL","NN","NNP","NNPS","NNS","PDT","POS","PRP","PRP$","RB","RBR","RBS","RP","SP","SYM","TO","UH","VB","VBD","VBG","VBN","VBP","VBZ","WDT","WP","WP$","WRB","ADD","NFP","GW","XX","BES","HVS","_SP"]
POS_LIST = ["ADJ", "ADP", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X", "SPACE"]
DEP_LIST = ["acl", "acomp", "advcl", "advmod", "agent", "amod", "appos", "attr", "aux", "auxpass", "case", "cc", "ccomp", "compound", "conj", "cop", "csubj", "csubjpass", "dative", "dep", "det", "dobj", "expl", "intj", "mark", "meta", "neg", "nn", "npmod", "npadvmod", "nsubj", "nsubjpass", "nummod","nmod", "oprd", "obj", "obl", "parataxis", "pcomp","predet", "pobj", "poss", "preconj", "prep", "prt", "punct",  "quantmod", "relcl", "ROOT", "xcomp"]
NER_LIST = ["PERSON", "NORP", "FAC", "ORG", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART", "LAW", "LANGUAGE", "DATE", "TIME", "PERCENT", "MONEY", "QUANTITY", "ORDINAL", "CARDINAL"]

In [32]:
# nlp = spacy.load('en_core_web_sm')

# nlp.tokenizer.vocab.morphology.tag_map
pos_tags = {}
for i,p in enumerate(TAG_LIST):
    pos_tags[p] = i

In [33]:
dep_tags = {}
for i,p in enumerate(DEP_LIST):
    dep_tags[p] = i

In [34]:
for i in range(len(train_jsons_aug)):
    train_jsons_aug[i]['pos_tags_num'] = [pos_tags[t] for t in train_jsons_aug[i]['pos_tags']]
    train_jsons_aug[i]['dep_tags_num'] = [dep_tags[t] for t in train_jsons_aug[i]['dep_label']]

In [35]:
for i in range(len(test_jsons_aug)):
    test_jsons_aug[i]['pos_tags_num'] = [pos_tags[t] for t in test_jsons_aug[i]['pos_tags']]
    test_jsons_aug[i]['dep_tags_num'] = [dep_tags[t] for t in test_jsons_aug[i]['dep_label']]

In [34]:
train_data.head()

,label,utterance
0,Rel,"Methods In this double-blind , placebo-control..."
1,NotRel,"In a preliminary study , the orally administer..."
2,Rel,"Methods In this double-blind , placebo-control..."
3,Rel,"Methods In this double-blind , placebo-control..."
4,Rel,"Methods In this double-blind , placebo-control..."


In [36]:
def prepare_features_aug(seq_2, var_dict, max_seq_length = 300, 
             zero_pad = True, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = seq_2

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    
    input_ids = tokens_a
    # input_ids = input_ids.squeeze(0)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [41]:
i=0
pos_tensor, _ = prepare_features_aug(train_df['pos_tags'][i], pos_tags, max_seq_length = 300, 
             zero_pad = True, include_CLS_token = True, include_SEP_token = True)
dep_tensor, _ = prepare_features_aug(train_df['dep_label'][i], dep_tags, max_seq_length = 300, 
             zero_pad = True, include_CLS_token = True, include_SEP_token = True)
tok_tensor,_= prepare_features(' '.join(train_df['tokens'][0]), max_seq_length = 300, 
             zero_pad = True, include_CLS_token = True, include_SEP_token = True)
ent_tensor,_ = prepare_features_aug(train_df['ent_type'][i], ent_dict, max_seq_length = 300, 
             zero_pad = True, include_CLS_token = True, include_SEP_token = True)

In [ ]:
# torch.cat((pos_tensor, dep_tensor,tok_tensor), dim=1).size()
# label
tok_tensor.size()

In [38]:
class Relations_Aug(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        utterance = self.data.tokens[index]
        label = self.data.label[index]
        pos_seq = self.data.pos_tags[index]
        dep_seq = self.data.dep_label[index]
        ent_seq = self.data.ent_type[index]
        pos_tensor, _ = prepare_features_aug(pos_seq, pos_tags, max_seq_length = 100, 
             zero_pad = True, include_CLS_token = True, include_SEP_token = True)
        dep_tensor, _ = prepare_features_aug(dep_seq, dep_tags, max_seq_length = 100, 
                     zero_pad = True, include_CLS_token = True, include_SEP_token = True)
        tok_tensor,_= prepare_features(utterance, max_seq_length = 100, 
                     zero_pad = True, include_CLS_token = True, include_SEP_token = True)
        ent_tensor,_ = prepare_features_aug(ent_seq, ent_dict, max_seq_length = 100, 
             zero_pad = True, include_CLS_token = True, include_SEP_token = True)
        X = torch.cat((pos_tensor, dep_tensor, tok_tensor, ent_tensor), dim=1)
        y = label_to_ix[label]

        return X,y
    
    def __len__(self):
        return self.len

In [39]:
def prepare_df(data):
    df = pd.DataFrame({'tokens':[], 'pos_tags': [], 'dep_label': [], 'label': [], 'ent_type': []})
    for index in tqdm_notebook(range(len(data))):
        k = 0
        ss1 = []
        ent_types = []
        for r in range(len(data[index]['relations'])):
            k = 0
            ss1 = []
            label = data[index]['relations'][r]['type']
            ent_types = []
            # print(label)
            while k<len(data[index]['tokens']):
                if k == data[index]['entities'][data[index]['relations'][r]['head']]['start']:
                    m = data[index]['entities'][data[index]['relations'][r]['head']]['end']
                    ss1.append('[s1]')
                    ss1.extend(data[index]['tokens'][k:m])
                    ss1.append('[e1]')
                    if k==m:
                        m+=1
                    for e in range(k,m):
                        ent_types.append(data[index]['entities'][data[index]['relations'][r]['head']]['type'])
                    k = m
                elif k == data[index]['entities'][data[index]['relations'][r]['tail']]['start']:
                    m = data[index]['entities'][data[index]['relations'][r]['tail']]['end']
                    ss1.append('[s2]')
                    ss1.extend(data[index]['tokens'][k:m])
                    ss1.append('[e2]')
                    if k==m:
                        m+=1
                    for e in range(k,m):
                        ent_types.append(data[index]['entities'][data[index]['relations'][r]['tail']]['type'])
                    k = m
                else:
                    ss1.append(data[index]['tokens'][k])
                    ent_types.append('O')
                    k+=1
            df0 = pd.concat([pd.Series(' '.join(ss1)), pd.Series([[pos_tags[t] for t in data[index]['pos_tags']]]), \
                             pd.Series([[dep_tags[t] for t in data[index]['dep_label']]]), 
                            pd.Series([label]), pd.Series([[ent_dict[t] for t in ent_types]])], axis = 1)
            df0.columns = df.columns
            df = pd.concat([df, df0])
            # df.loc[index,'tokens'] = ss1
            # df.loc[index,'pos_tags'] = data[index]['pos_tags']
            # df.loc[index,'dep_label'] = data[index]['dep_label']
            # df.loc[i,'label'] = label
    return df.reset_index()

In [40]:
train_df = prepare_df(train_jsons_aug)
test_df = prepare_df(test_jsons_aug)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/983 [00:00<?, ?it/s]

  0%|          | 0/226 [00:00<?, ?it/s]

In [58]:
train_data = train_df[['label','tokens']]
train_data.columns = ['label','utterance']

test_data = test_df[['label','tokens']]
test_data.columns = ['label','utterance']

In [59]:
test_df.label.value_counts()

NotRel    898
Rel       470
Name: label, dtype: int64

In [60]:
train_df.label.value_counts()

NotRel    4286
Rel       1812
Name: label, dtype: int64

In [58]:
ent_dict

{'ade': 0,
 'drug': 1,
 'duration': 2,
 'mm_hg': 3,
 '6mwd': 4,
 'reason': 5,
 'stength': 6,
 'frequency': 7,
 'rate': 8,
 'meanpap': 9,
 'route': 10,
 'fc': 11,
 'dosage': 12,
 'strength': 13,
 '*': 14,
 'percent_rate': 15,
 'form': 16,
 'nt-probnp': 17,
 'death_rate': 18,
 'death': 19,
 'prev_treat': 20,
 '_': 21,
 'progression': 22,
 'hospitalization': 23,
 'unsatisfactory_response': 24,
 'O': 25}

In [61]:
train_df.head()

,index,tokens,pos_tags,dep_label,label,ent_type
0,0,Figure 2A demonstrates a [s1] signiﬁcant drop ...,"[25, 25, 46, 14, 19, 25, 18, 14, 26, 19, 25, 1...","[13, 30, 48, 20, 5, 21, 43, 20, 33, 5, 40, 43,...",Rel,"[25, 25, 25, 25, 0, 0, 0, 0, 0, 0, 0, 25, 25, ..."
1,0,Figure 2A demonstrates a [s1] signiﬁcant drop ...,"[25, 25, 46, 14, 19, 25, 18, 14, 26, 19, 25, 1...","[13, 30, 48, 20, 5, 21, 43, 20, 33, 5, 40, 43,...",NotRel,"[25, 25, 25, 25, 0, 0, 0, 0, 0, 0, 0, 25, 25, ..."
2,0,Figure 2A demonstrates a [s1] signiﬁcant drop ...,"[25, 25, 46, 14, 19, 25, 18, 14, 26, 19, 25, 1...","[13, 30, 48, 20, 5, 21, 43, 20, 33, 5, 40, 43,...",NotRel,"[25, 25, 25, 25, 0, 0, 0, 0, 0, 0, 0, 25, 25, ..."
3,0,Figure 2A demonstrates a [s1] signiﬁcant drop ...,"[25, 25, 46, 14, 19, 25, 18, 14, 26, 19, 25, 1...","[13, 30, 48, 20, 5, 21, 43, 20, 33, 5, 40, 43,...",NotRel,"[25, 25, 25, 25, 0, 0, 0, 0, 0, 0, 0, 25, 25, ..."
4,0,Figure 2A demonstrates a [s1] signiﬁcant drop ...,"[25, 25, 46, 14, 19, 25, 18, 14, 26, 19, 25, 1...","[13, 30, 48, 20, 5, 21, 43, 20, 33, 5, 40, 43,...",Rel,"[25, 25, 25, 25, 0, 0, 0, 0, 0, 0, 0, 25, 25, ..."


In [55]:
data[index]['relations'][1]['head']

2

In [54]:
data[index]['entities'][]

[{'type': 'drug', 'start': 9, 'end': 10},
 {'type': 'reason', 'start': 21, 'end': 24},
 {'type': 'drug', 'start': 11, 'end': 12}]

In [42]:
training_set_aug = Relations_Aug(train_df)
testing_set_aug = Relations_Aug(test_df)

In [43]:
training_set_aug.__getitem__(0)[0].size()

torch.Size([1, 400])

In [44]:
torch.cat((pos_tensor, dep_tensor), dim=1).size()

torch.Size([1, 600])

In [45]:
pos_tensor.squeeze(1).size()

torch.Size([1, 300])

In [46]:
params = {'batch_size': 5,
          'drop_last': False,
          'shuffle': True,
          'num_workers': 1}

In [47]:
sampler = torch.utils.data.sampler.BatchSampler(
    torch.utils.data.sampler.RandomSampler(training_set_aug),
    batch_size=1,
    drop_last=False)

In [48]:
training_loader_aug = DataLoader(training_set_aug, **params)
testing_loader_aug = DataLoader(testing_set_aug, **params)

In [49]:
loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-04
optimizer = optim.SGD(params =  model.parameters(), lr=learning_rate)

In [50]:
## Test Forward Pass
inp = training_set_aug.__getitem__(0)[0].cuda()
output = model(inp)[0]
print(inp.shape, output.shape)

torch.Size([1, 400]) torch.Size([1, 2])


In [ ]:
# for data, label in training_loader_aug:
#     print(data.squeeze(1).size(), label.size())

In [64]:
max_epochs = 20
model = model.train()
for epoch in tqdm_notebook(range(max_epochs)):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader_aug):
        optimizer.zero_grad()
        sent = sent.squeeze(1)
        if torch.cuda.is_available():
          sent = sent.cuda()
          label = label.cuda()
        output = model.forward(sent)[0]
        _, predicted = torch.max(output, 1)
        
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        if i%100 == 0:
            correct = 0
            total = 0
            for sent, label in testing_loader_aug:
                sent = sent.squeeze(1)
                if torch.cuda.is_available():
                  sent = sent.cuda()
                  label = label.cuda()
                output = model.forward(sent)[0]
                _, predicted = torch.max(output.data, 1)
                total += label.size(0)
                correct += (predicted.cpu() == label.cpu()).sum()
            accuracy = 100.00 * correct.numpy() / total
            if accuracy>latest_best_score and accuracy>0.7:
                latest_best_score = accuracy
                torch.save(model.state_dict(), 'model_roberta_aug_'+str(epoch)+'_'+ str(uuid4())+'.pth')

            print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/20 [00:00<?, ?it/s]

EPOCH -- 0
Iteration: 0. Loss: 0.2761631906032562. Accuracy: 65.5701754385965%
Iteration: 100. Loss: 0.5093924403190613. Accuracy: 65.71637426900585%
Iteration: 200. Loss: 0.6496957540512085. Accuracy: 65.64327485380117%
Iteration: 300. Loss: 0.7034847140312195. Accuracy: 65.78947368421052%
Iteration: 400. Loss: 0.38037270307540894. Accuracy: 65.71637426900585%
Iteration: 500. Loss: 0.6143695116043091. Accuracy: 65.64327485380117%
Iteration: 600. Loss: 0.889144778251648. Accuracy: 65.71637426900585%
Iteration: 700. Loss: 0.6716240644454956. Accuracy: 65.5701754385965%
Iteration: 800. Loss: 0.7250439524650574. Accuracy: 65.71637426900585%
Iteration: 900. Loss: 0.9290422201156616. Accuracy: 65.78947368421052%
Iteration: 1000. Loss: 0.8898609280586243. Accuracy: 65.13157894736842%
Iteration: 1100. Loss: 0.5079776048660278. Accuracy: 65.71637426900585%
Iteration: 1200. Loss: 0.7306002378463745. Accuracy: 65.5701754385965%
EPOCH -- 1
Iteration: 0. Loss: 0.5137914419174194. Accuracy: 65.4970

In [30]:
# for data in training_loader_aug:
#     print(data)

In [50]:
torch.save(model.state_dict(), 'model_4f_vars'+ str(uuid4())+'.pth')

In [68]:
model_path = 'model_roberta_aug_13_599ce3ff-c7e3-4e99-924b-16a3755a6037.pth'

In [69]:
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [66]:
def get_reply(data,index):
    model.eval()
    utterance = ' '.join(data.tokens[index])
    pos_seq = data.pos_tags[index]
    dep_seq = data.dep_label[index]
    ent_seq = data.ent_type[index]
    pos_tensor, _ = prepare_features_aug(pos_seq, pos_tags, max_seq_length = 100, 
         zero_pad = True, include_CLS_token = True, include_SEP_token = True)
    dep_tensor, _ = prepare_features_aug(dep_seq, dep_tags, max_seq_length = 100, 
                 zero_pad = True, include_CLS_token = True, include_SEP_token = True)
    tok_tensor,_= prepare_features(utterance, max_seq_length = 100, 
                 zero_pad = True, include_CLS_token = True, include_SEP_token = True)
    ent_tensor,_ = prepare_features_aug(ent_seq, ent_dict, max_seq_length = 100, 
         zero_pad = True, include_CLS_token = True, include_SEP_token = True)
    X = torch.cat((pos_tensor, dep_tensor, tok_tensor, ent_tensor), dim=1)

    if torch.cuda.is_available():
        X = X.cuda()
    output = model(X)[0]
    _, pred_label = torch.max(output.data, 1)
    prediction=list(label_to_ix.keys())[pred_label]
    return prediction

In [25]:
inf_jsons_aug = json.load(open('pah/data/articles_inf_aa_aug.json'))


In [30]:
inf_jsons_aug[10]['entities']

[{'type': 'duration', 'start': 7, 'end': 9}]

In [65]:
inf_df = prepare_df(inf_jsons_aug)
inf_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/30348 [00:00<?, ?it/s]

,index,tokens,pos_tags,dep_label,label,ent_type
0,0,Comparison of hemodynamic parameters in treatm...,"[25, 18, 19, 28, 18, 19, 12, 42, 28, 18, 19, 1...","[48, 43, 5, 40, 43, 5, 11, 14, 40, 43, 5, 5, 4...",NotRel,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 5, 5,..."
1,0,Comparison of hemodynamic parameters in treatm...,"[25, 18, 19, 28, 18, 19, 12, 42, 28, 18, 19, 1...","[48, 43, 5, 40, 43, 5, 11, 14, 40, 43, 5, 5, 4...",NotRel,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 5, 5,..."
2,0,E-mail address : nazzareno.galie @ unibo.it BA...,"[25, 25, 2, 25, 18, 26, 26, 2, 25, 18, 19, 19,...","[13, 48, 45, 19, 45, 13, 48, 45, 48, 43, 5, 5,...",NotRel,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2..."
3,0,E-mail address : nazzareno.galie @ unibo.it BA...,"[25, 25, 2, 25, 18, 26, 26, 2, 25, 18, 19, 19,...","[13, 48, 45, 19, 45, 13, 48, 45, 48, 43, 5, 5,...",NotRel,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2..."
4,0,E-mail address : nazzareno.galie @ unibo.it BA...,"[25, 25, 2, 25, 18, 26, 26, 2, 25, 18, 19, 19,...","[13, 48, 45, 19, 45, 13, 48, 45, 48, 43, 5, 5,...",NotRel,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2..."


In [70]:
inf_rel = []
for i in tqdm_notebook(range(inf_df.shape[0])):
    inf_rel.append(get_reply(inf_df, i))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/74220 [00:00<?, ?it/s]

In [41]:
test_rel = []
for i in tqdm_notebook(range(test_df.shape[0])):
    test_rel.append(get_reply(test_df, i))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1368 [00:00<?, ?it/s]

In [42]:
test_rel_df = pd.DataFrame(test_rel)
test_rel_df.columns = ['Relation']

In [71]:
inf_rel_df = pd.DataFrame(inf_rel)
inf_rel_df.columns = ['Relation']

In [43]:
test_rel_df.Relation.value_counts()

NotRel    1091
Rel        277
Name: Relation, dtype: int64

In [44]:
test_df.label.value_counts()

NotRel    898
Rel       470
Name: label, dtype: int64

In [77]:
inf_rel_df.Relation.value_counts()

NotRel    74220
Name: Relation, dtype: int64

In [73]:
inf_df = pd.concat([inf_df, inf_rel_df], axis = 1)
inf_df.head()

,index,tokens,pos_tags,dep_label,label,ent_type,Relation
0,0,Comparison of hemodynamic parameters in treatm...,"[25, 18, 19, 28, 18, 19, 12, 42, 28, 18, 19, 1...","[48, 43, 5, 40, 43, 5, 11, 14, 40, 43, 5, 5, 4...",NotRel,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 5, 5,...",NotRel
1,0,Comparison of hemodynamic parameters in treatm...,"[25, 18, 19, 28, 18, 19, 12, 42, 28, 18, 19, 1...","[48, 43, 5, 40, 43, 5, 11, 14, 40, 43, 5, 5, 4...",NotRel,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 5, 5,...",NotRel
2,0,E-mail address : nazzareno.galie @ unibo.it BA...,"[25, 25, 2, 25, 18, 26, 26, 2, 25, 18, 19, 19,...","[13, 48, 45, 19, 45, 13, 48, 45, 48, 43, 5, 5,...",NotRel,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...",NotRel
3,0,E-mail address : nazzareno.galie @ unibo.it BA...,"[25, 25, 2, 25, 18, 26, 26, 2, 25, 18, 19, 19,...","[13, 48, 45, 19, 45, 13, 48, 45, 48, 43, 5, 5,...",NotRel,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...",NotRel
4,0,E-mail address : nazzareno.galie @ unibo.it BA...,"[25, 25, 2, 25, 18, 26, 26, 2, 25, 18, 19, 19,...","[13, 48, 45, 19, 45, 13, 48, 45, 48, 43, 5, 5,...",NotRel,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...",NotRel


In [61]:
inf_df[inf_df['Relation']=='Rel']

,index,tokens,pos_tags,dep_label,label,ent_type,Relation
9,0,"[METHODS, :, Patients, with, [s1], PAH, [e1], ...","[NNP, :, NNS, IN, NNP, WP, VBD, NN, JJ, CC, VB...","[ROOT, punct, appos, prep, pobj, nsubj, relcl,...",NotRel,"[O, O, O, O, reason, O, O, O, O, O, O, O, O, O...",Rel
10,0,"[METHODS, :, Patients, with, [s1], PAH, [e1], ...","[NNP, :, NNS, IN, NNP, WP, VBD, NN, JJ, CC, VB...","[ROOT, punct, appos, prep, pobj, nsubj, relcl,...",NotRel,"[O, O, O, O, reason, O, O, O, O, O, O, O, O, O...",Rel
13,0,"[METHODS, :, Patients, with, PAH, who, were, t...","[NNP, :, NNS, IN, NNP, WP, VBD, NN, JJ, CC, VB...","[ROOT, punct, appos, prep, pobj, nsubj, relcl,...",NotRel,"[O, O, O, O, O, O, O, O, O, O, O, O, drug, O, ...",Rel
14,0,"[METHODS, :, Patients, with, [s2], PAH, [e2], ...","[NNP, :, NNS, IN, NNP, WP, VBD, NN, JJ, CC, VB...","[ROOT, punct, appos, prep, pobj, nsubj, relcl,...",NotRel,"[O, O, O, O, reason, O, O, O, O, O, O, O, O, O...",Rel
17,0,"[METHODS, :, Patients, with, [s2], PAH, [e2], ...","[NNP, :, NNS, IN, NNP, WP, VBD, NN, JJ, CC, VB...","[ROOT, punct, appos, prep, pobj, nsubj, relcl,...",NotRel,"[O, O, O, O, reason, O, O, O, O, O, O, O, O, O...",Rel
...,...,...,...,...,...,...,...
74181,0,"[In, this, trial, ,, a, total, of, 185, PAH, p...","[IN, DT, NN, ,, DT, NN, IN, CD, NNP, NNS, IN, ...","[prep, det, pobj, punct, det, nsubjpass, prep,...",NotRel,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",Rel
74184,0,"[In, this, trial, ,, a, total, of, 185, PAH, p...","[IN, DT, NN, ,, DT, NN, IN, CD, NNP, NNS, IN, ...","[prep, det, pobj, punct, det, nsubjpass, prep,...",NotRel,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",Rel
74185,0,"[In, this, trial, ,, a, total, of, 185, PAH, p...","[IN, DT, NN, ,, DT, NN, IN, CD, NNP, NNS, IN, ...","[prep, det, pobj, punct, det, nsubjpass, prep,...",NotRel,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",Rel
74188,0,"[In, this, trial, ,, a, total, of, 185, PAH, p...","[IN, DT, NN, ,, DT, NN, IN, CD, NNP, NNS, IN, ...","[prep, det, pobj, punct, det, nsubjpass, prep,...",NotRel,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",Rel


In [75]:
train_df.label.value_counts()

NotRel    932
Rel       179
Name: label, dtype: int64

In [ ]:
inf_df[['tokens', 'Relation']].to_csv('inf_model_roberta_best.csv')

In [76]:
# inf_df= pd_read_csv('inf_model_roberta_best.csv')
inf_df[200:250].to_csv('inf_model_rob_best_sample_50.csv', index = False)